For linear initial value problems, stability analysis can be performed by:

1.  Computing the eigenvalues $\lambda_j$ of the ODE system.
2.  Determining the maximal timestep $h$ such that $h \lambda_j$ lies inside the absolute stability region of the time stepping method.

This is particularly useful when applying the method of lines to a PDE system; in this case, the eigenvalues come from the semi-discretization of the PDE.  Here we show how this can be done in NodePy with some simple examples.

In [ ]:
from matplotlib import pyplot as plt
from nodepy import rk, semidisc

First we load the classical method of Kutta and the simple first-order upwind semi-discretization of the advection equation (with periodic boundary conditions.

In [ ]:
rk4 = rk.loadRKM('RK44')

upwind = semidisc.upwind_advection_matrix(100,dx=0.1)

Now we can use the method `linearly_stable_step_size` to automatically compute the maximum absolutely stable step size for this combination.  As a bonus, the function shows a plot of the absolute stability region and the (maximally) scaled spectrum.

In [ ]:
rk.linearly_stable_step_size(rk4,upwind)

We can do the same for another linear spatial discretization.

In [ ]:
spectral = semidisc.load_semidisc('spectral difference advection',order=6)
rk.linearly_stable_step_size(rk4,spectral.L)

For a nonlinear discretization, like WENO, a simple approach is to consider the eigenvalues of one or more linearizations.  Here we use the linearization given in the limit of a very smooth function.

In [ ]:
weno5 = semidisc.weno5_linearized_matrix(100)
rk.linearly_stable_step_size(rk4,weno5)

The next example uses a semi-discretization of an advection-diffusion PDE.  For this stiff PDE, RK4 requires a very small step size and would be inefficient.

In [ ]:
centered_diff = semidisc.centered_advection_diffusion_matrix(10.,1.,N=100)
rk.linearly_stable_step_size(rk4,centered_diff)

A better choice would be a method with good real-axis stability, like the Runge-Kutta-Chebyshev methods.  Let's try one.

In [ ]:
rkc = rk.RKC2(10)
rk.linearly_stable_step_size(rkc,centered_diff)

Unfortunately, this doesn't work as well as hoped because the PDE in question has a small but non-zero convective component.  We can modify the RKC method in order to accommodate this:

In [ ]:
rkc_damped = rk.RKC2(10, epsilon=0.2)
rk.linearly_stable_step_size(rkc_damped,centered_diff)

Notice that the step size allowed is 10x larger than with the undamped RKC method, and over 20x larger than with the RK4 method.

How does the real-axis stability compare between the damped and undamped RKC methods?

In [ ]:
rkc.real_stability_interval()

In [ ]:
rkc_damped.real_stability_interval()

As a last example, here's an interactive widget that lets you design an RKC method (choosing the number of stages and the damping parameter) and see the resulting allowed step size.  Experiment with it.  How much damping do you need?  Does it depend on the number of stages?  What method is most efficient?

In [ ]:
from ipywidgets import interact, IntSlider, FloatSlider

In [ ]:
def plot_rkc2_stability_region(s=5,epsilon=0.):
    rkc = rk.RKC2(s, epsilon)
    h = rk.linearly_stable_step_size(rkc,centered_diff, tol=1.e-13)
    print('Maximum stable step size: {:.3e}'.format(h))
    plt.show()

In [ ]:
interact(plot_rkc2_stability_region,
         s=IntSlider(min=1,max=20,value=5),
         epsilon=FloatSlider(min=0.,max=2.,step=0.01,value=0.));